In [666]:
import pandas as pd
import numpy as np

In [667]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")




In [668]:
movies=movies.merge(credits,on="title")

In [669]:
movies=movies[["title","movie_id","genres","keywords","overview","cast","crew",]]


In [670]:
movies.dropna(inplace=True)

In [671]:
import ast


In [672]:
def convert(obj):
    lst=[]
    for i in ast.literal_eval(obj):
        lst.append(i["name"].replace(" ",""))
    return lst
    

In [673]:
movies["genres"]=movies.genres.apply(convert)
movies["keywords"]=movies.keywords.apply(convert)


In [674]:
def convert_cast(obj):
    lst=[]
    for i in ast.literal_eval(obj):
        if len(lst)>=3:  # picking top 3 characters
            break
        lst.append(i["name"].replace(" ",""))
    return lst
    

In [675]:
movies["cast"]=movies.cast.apply(convert_cast)


In [676]:
def convert_crew(obj):
    lst=[]
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            lst.append(i["name"].replace(" ",""))
            break
    return lst
    

In [677]:
movies.crew=movies.crew.apply(convert_crew)

In [678]:
# movies["title"]=movies["title"].apply(lower_case)
movies["tags"]=movies.overview.apply(lambda x:x.split() )+movies.genres+movies.keywords+movies.cast+movies.crew

In [679]:

movies.tags=movies.tags.apply(lambda obj:" ".join(obj))
movies.tags=movies.tags.apply(lambda x: x.lower())


In [680]:
                                
movies.tags[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [681]:
new_df=movies[["title","movie_id","tags"]]
new_df

,title,movie_id,tags
0,Avatar,19995,"in the 22nd century, a paraplegic marine is di..."
1,Pirates of the Caribbean: At World's End,285,"captain barbossa, long believed to be dead, ha..."
2,Spectre,206647,a cryptic message from bond’s past sends him o...
3,The Dark Knight Rises,49026,following the death of district attorney harve...
4,John Carter,49529,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,El Mariachi,9367,el mariachi just wants to play his guitar and ...
4805,Newlyweds,72766,a newlywed couple's honeymoon is upended by th...
4806,"Signed, Sealed, Delivered",231617,"""signed, sealed, delivered"" introduces a dedic..."
4807,Shanghai Calling,126186,when ambitious new york attorney sam is sent t...


# Vectorizatuon

In [682]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english",binary=True)

In [683]:
vector=cv.fit_transform(new_df["tags"]).toarray()

In [684]:
cv.get_feature_names_out()
vector.shape

(4806, 5000)

# Stemming

In [685]:
import nltk

In [686]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [687]:
def stemming(text):
    L=[]
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)

In [688]:
new_df.iloc[0]["tags"]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [689]:
'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'


'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

# Similarity

In [690]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
similarity.shape

(4806, 4806)

In [691]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1192, 0.254000254000381),
 (61, 0.2351585405008862),
 (47, 0.22416096662402304),
 (260, 0.21997067253202998),
 (322, 0.21561438627179413)]

In [693]:
def recommended(movie):
    movie_index=new_df[new_df["title"]==movie].index[0]
    indexes=sorted(list(enumerate(similarity[movie_index])),reverse=True,key=lambda x:x[1])[1:6]
    l=[]
    for i in indexes:
        print(i)
        l.append(new_df.iloc[i[0]].title)
    print(l)
    
recommended("Batman Begins")
    
    

(3, 0.33686076842660756)
(65, 0.3368607684266075)
(1361, 0.30414953233623676)
(1360, 0.280717307022173)
(210, 0.2712125199968458)
['The Dark Knight Rises', 'The Dark Knight', 'Batman', 'Batman', 'Batman & Robin']


In [694]:

import pickle

In [696]:
pickle.dump(new_df,open("movies.pkl","wb"))


In [697]:
pickle.dump(similarity,open("similarity.pkl","wb"))
